In [1]:
# 导入os模块
import os
# 设置环境变量，将可见的GPU设置为0,1,2,3,4,5,6
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6"
# 导入pandas模块
import pandas as pd 
# 设置pandas显示的最大列数为None，即不限制显示的列数
pd.set_option('display.max_columns', None)
# 导入optuna模块
import optuna
# 导入math模块
import math
# 导入numpy模块
import numpy as np 
# 导入seaborn模块
import seaborn as sns 
# 从autogluon.common.savers模块中导入save_pd函数
from autogluon.common.savers import save_pd
# 从autogluon.tabular模块中导入TabularPredictor和TabularDataset类
from autogluon.tabular import TabularPredictor, TabularDataset
# 导入warnings模块
import warnings 
# 忽略警告信息
warnings.filterwarnings("ignore")
# 再次设置pandas显示的最大列数为None，即不限制显示的列数
pd.set_option('display.max_columns', None)

/home/zhenghao/.conda/envs/light_new/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'autogluon'

In [2]:
# 创建训练集数据集
tr_d = TabularDataset('/home/zhenghao/kaggle/train.csv')
# 创建测试集数据集
te_d = TabularDataset('/home/zhenghao/kaggle/test.csv')
# 删除训练集数据集中的id列
tr_d = tr_d.drop("id",axis=1)


In [3]:
tr_d.head()


,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [4]:
te_d.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148


In [5]:
params = {
    
    "XGB": {
        
        "colsample_bytree": 0.516652313273348,
        "enable_categorical": True,
        "learning_rate": 0.015,
        "max_depth": 9,
        "min_child_weight": 0.8567068904025429,
        "max_bin": 262143,
        "n_estimators": 10000,
        'tree_method': 'hist', 
        'device': 'cuda'
        
          },
    
    'GBM': {
          
        'n_estimators': 1190,
        'learning_rate': 0.22952000374471332,
        'max_depth': 13, 
        'reg_alpha': 8.200152384535924,
        'reg_lambda': 4.285393733702208, 
        'num_leaves': 100, 
        'subsample': 0.6497981764924947, 
        'colsample_bytree': 0.37368304607248115
           
           } 

}

In [6]:
print(tr_d.info())
print(tr_d.describe))


<class 'autogluon.core.dataset.TabularDataset'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Gender                object 
 1   Age                   int64  
 2   Driving_License       int64  
 3   Region_Code           float64
 4   Previously_Insured    int64  
 5   Vehicle_Age           object 
 6   Vehicle_Damage        object 
 7   Annual_Premium        float64
 8   Policy_Sales_Channel  float64
 9   Vintage               int64  
 10  Response              int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 965.5+ MB
None
                Age  Driving_License   Region_Code  Previously_Insured  \
count  1.150480e+07     1.150480e+07  1.150480e+07        1.150480e+07   
mean   3.838356e+01     9.980220e-01  2.641869e+01        4.629966e-01   
std    1.499346e+01     4.443120e-02  1.299159e+01        4.986289e-01   
min    2.000000e+01     0.000000e+00  0.000000e+00     

In [7]:
# 定义要包含的模型类型
included_models = ["GBM", "XGB",]

# 创建TabularPredictor对象，并设置参数
PREDICTOR_AUTO = TabularPredictor(label = 'Response'
                                  ,eval_metric = 'roc_auc',).fit(
        
        # 设置训练数据
        train_data = tr_d,
        
        # 设置超参数
        hyperparameters = params,
        
        # 设置时间限制
        time_limit = 3600*6,  
        
        # 设置预设参数
        presets='best_quality',
        
        # 设置训练参数
        ag_args_fit={'num_gpus': 6} ,
    
        # 设置包含的模型类型
        included_model_types = included_models,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240708_125826"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #46-Ubuntu SMP Fri Jul 10 00:24:02 UTC 2020
CPU Count:          48
Memory Avail:       197.37 GB / 251.56 GB (78.5%)
Disk Space Avail:   1801.97 GB / 3509.42 GB (51.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fi

KeyboardInterrupt: 

(_dystack pid=381141) 	Warning: Exception caused LightGBM_BAG_L1 to fail during training (ImportError)... Skipping this model.
(_dystack pid=381141) 		ray::_ray_fit() (pid=400039, ip=192.168.100.200)
(_dystack pid=381141) ModuleNotFoundError: No module named 'lightgbm'
(_dystack pid=381141) 
(_dystack pid=381141) During handling of the above exception, another exception occurred:
(_dystack pid=381141) 
(_dystack pid=381141) ray::_ray_fit() (pid=400039, ip=192.168.100.200)
(_dystack pid=381141)   File "/home/zhenghao/.conda/envs/match/lib/python3.11/site-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 402, in _ray_fit
(_dystack pid=381141)     fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
(_dystack pid=381141)   File "/home/zhenghao/.conda/envs/match/lib/python3.11/site-packages/autogluon/core/models/abstract/abstract_model.py", line 856, in fit
(_dystack pid=381141)     out = self._

In [ ]:
PREDICTOR_AUTO.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
